<a href="https://colab.research.google.com/github/aledima00/Project4_SemSeg_AML2024/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 4 - Semantic Segmentation

First let's download dataset, that is already split in "Train", "Test" and "Val" modules

In [ ]:
!pip install skimage
!pip install icecream

In [2]:
import os

import logging
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import glob
import numpy as np
from enum import Enum
from PIL import Image
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [2]:
!wget -O "train.zip" "https://zenodo.org/records/5706578/files/Train.zip?download=1"
!unzip "train.zip" | tail -n 3

--2024-12-17 15:50:56--  https://zenodo.org/records/5706578/files/Train.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.45.92, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4021669263 (3.7G) [application/octet-stream]
Saving to: ‘train.zip’

train.zip           100%[===================>]   3.75G  12.0MB/s    in 5m 25s  

2024-12-17 15:56:22 (11.8 MB/s) - ‘train.zip’ saved [4021669263/4021669263]

  inflating: Train/Urban/masks_png/2519.png  
  inflating: Train/Urban/masks_png/2520.png  
  inflating: Train/Urban/masks_png/2521.png  


In [32]:
!wget -O "val.zip" "https://zenodo.org/records/5706578/files/Val.zip?download=1"
!unzip "val.zip" | tail -n 3

--2024-12-17 16:20:17--  https://zenodo.org/records/5706578/files/Val.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2425958254 (2.3G) [application/octet-stream]
Saving to: ‘val.zip’

val.zip             100%[===================>]   2.26G  18.3MB/s    in 2m 32s  

2024-12-17 16:22:49 (15.2 MB/s) - ‘val.zip’ saved [2425958254/2425958254]

  inflating: Val/Urban/masks_png/4188.png  
  inflating: Val/Urban/masks_png/4189.png  
  inflating: Val/Urban/masks_png/4190.png  


In [36]:
!wget -O "Test.zip" "https://zenodo.org/records/5706578/files/Test.zip?download=1"
!unzip "Test.zip" | tail -n 3

--2024-12-17 16:30:40--  https://zenodo.org/records/5706578/files/Test.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3126023212 (2.9G) [application/octet-stream]
Saving to: ‘Test.zip’

Test.zip              3%[                    ] 109.44M   650KB/s    eta 71m 56s^C
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of Test.zip or
        Test.zip.zip, and cannot find Test.zip.ZIP, period.
Archive:  Test.zip


## Data Preparation

Create Dataset class and filter urban pictures...

In [3]:
DBG = False

DEVICE = "cuda"
TRAIN_PATH = "Train"
TEST_PATH = "Test"
VAL_PATH = "Val"

# All this param can be change!

NUM_CLASSES = 7
BATCH_SIZE = 2 if DBG else 128
LR = 0.001           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 4e-5  # Regularization, you can keep this at the default
NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
STEP_SIZE = [25, 75, 150] # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down




In [4]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # Normalize as per ImageNet stats
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # Normalize as per ImageNet stats
])

In [5]:
COLOR_MAP = {
    "IGNORE":0,
    "Background":1,
    "Building":2,
    "Road":3,
    "Water":4,
    "Barren":5,
    "Forest":6,
    "Agricultural":7,
}

LABELS = {key:i for key,i in enumerate(COLOR_MAP.values()) }
LABELS

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}

In [ ]:
!pip install skimage

In [13]:
from skimage.io import imread


class DataType(Enum):
    RURAL = 0
    URBAN = 1

class LoveDA(Dataset):
    def __init__(self, basedir, data_type:DataType, transforms=None, target_transform=None):
        #super(LoveDA, self).__init__(basedir, transforms, target_transform)
        if data_type == DataType.RURAL:
            self.base_path = os.path.join(basedir, "Rural")
        else: #data_type == DataType.URBAN:
            self.base_path = os.path.join(basedir, "Urban")


        # list of integers that identifies paths relative to both images_png and masks_png
        self.int_pathrefs = os.listdir(os.path.join(self.base_path, "images_png"))
        self.int_pathrefs = list(int(st.split(".")[0]) for st in self.int_pathrefs)
        self.transforms = transforms

    def __getitem__(self, idx):
      assert idx < len(self), 'Index out of range'
      image_path = os.path.join(self.base_path, "images_png", str(self.int_pathrefs[idx]) + ".png")
      mask_path = os.path.join(self.base_path, "masks_png", str(self.int_pathrefs[idx]) + ".png")
      image = imread(image_path)
      mask = imread(mask_path)


      if self.transforms is not None:
          image = self.transforms(image)
          mask = self.transforms(mask)

      image,mask = torch.tensor(image),torch.tensor(mask)
      labels = torch.zeros((224,224),dtype=np.int8)

      for key, value in LABELS.items():
        indexes = (mask == value)
        labels[indexes] = key
      return image, labels

    def __len__(self):
        return len(self.int_pathrefs)


In [14]:
# Urban Datasets
urban_val = LoveDA(VAL_PATH, DataType.URBAN, transforms=eval_transform)
urban_train = LoveDA(VAL_PATH, DataType.URBAN, transforms=train_transform)

# Urban Dataloader
urban_train_dataloader = DataLoader(urban_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
urban_val_dataloader = DataLoader(urban_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

Download the pretrained model with *Resnet101* as backbone

Get DeepLabv2 implementation:

In [5]:
!git clone https://github.com/Gabrysse/MLDL2024_project1.git

fatal: destination path 'MLDL2024_project1' already exists and is not an empty directory.


In [8]:
import sys
sys.path.append("/content/MLDL2024_project1/")
from MLDL2024_project1.models.deeplabv2 import deeplabv2

In [9]:
import gdown
gdown.download("https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v", "pretrain-weights.pth", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v
From (redirected): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v&confirm=t&uuid=9c145447-f117-4931-8a16-0e53aa720c6c
To: /content/pretrain-weights.pth
100%|██████████| 177M/177M [00:04<00:00, 41.3MB/s]


'pretrain-weights.pth'

In [10]:
model = deeplabv2.get_deeplab_v2(num_classes=7,pretrain=True,pretrain_model_path="pretrain-weights.pth")

Deeplab pretraining loading...


/content/MLDL2024_project1/models/deeplabv2/deeplabv2.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(pretrain_model_path)


## Optimizer, Loss, ecc.

In [15]:
# enable validation during training
validate = True

model.train(True)
model.multi_level = False # ask in class
for params in model.get_1x_lr_params_no_scale():
  params.requires_grad = False # no training in Backbone
for params in model.get_10x_lr_params():
  params.requires_grad = True # training in classifiers


model = model.to(DEVICE) # switch to GPU

#Loss (as said in DeepLabv2 docs)
loss_fn = nn.CrossEntropyLoss()

#Opt
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer_fn = optim.Adam(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY, eps=1e-4)

#Scheduler
optim_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=STEP_SIZE, gamma=GAMMA)


## Training Loop

In [16]:
import warnings
from torch.backends import cudnn
from icecream import ic
from colorama import Fore,Back,Style
warnings.filterwarnings('ignore')

train_iter = 0
val_iter = 0

trainSamples = len(urban_train) - (len(urban_train) % BATCH_SIZE)
val_samples = len(urban_val)
iterPerEpoch = len(urban_train_dataloader)
val_steps = len(urban_val_dataloader)

cudnn.benchmark
model_checkpoint = "model" #name

model.train(True)
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    IoU = 0

    for i, (inputs, targets) in enumerate(urban_train_dataloader):
        train_iter += 1
        optimizer_fn.zero_grad()

        # feeds in model
        inputs = inputs.to(DEVICE)
        labels = targets.to(device=DEVICE, dtype=torch.int64)

        output_logits,_,_ = model(inputs)

        # compute loss
        loss = loss_fn(output_logits, labels)

        # backward loss and optimizer step
        loss.backward()
        optimizer_fn.step()

        #compute the training accuracy
        _, predicted = torch.max(output_logits.data, 1)
        print(labels)
        print(predicted)

        intersection = torch.logical_and(labels, predicted)
        union = torch.logical_or(labels, predicted)
        print(intersection)
        print(union)
        IoU += torch.sum(intersection).item() / torch.sum(union).item()

        step_loss = loss.data.item()
        epoch_loss += step_loss
    avg_loss = epoch_loss/iterPerEpoch
    print(f"IoU:{IoU}, samples:{trainSamples}")
    avg_IoU = (IoU / trainSamples)
    print(Fore.GREEN + 'Val: Epoch = {} | Loss {:.3f} | mean-IoU = {:.3f}'.format(epoch + 1, avg_loss, avg_IoU))
    """
    #train_logger.add_epoch_data(epoch+1, trainAccuracy, avg_loss)

    if validate:
        if (epoch+1) % 1 == 0:
            model.train(False)
            val_loss_epoch = 0
            numCorr = 0
            for j, (inputs, targets) in enumerate(val_loader):
                val_iter += 1
                inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
                labelVariable = targets.to(DEVICE)

                output_label, _ = model(inputVariable)
                val_loss = loss_fn(output_label, labelVariable)

                val_loss_step = val_loss.data.item()
                val_loss_epoch += val_loss_step
                _, predicted = torch.max(output_label.data, 1)
                numCorr += torch.sum(predicted == labelVariable.data).data.item()
                #val_logger.add_step_data(val_iter, numCorr, val_loss_step)

            val_accuracy = (numCorr / val_samples) * 100
            avg_val_loss = val_loss_epoch / val_steps

            print(Fore.GREEN + 'Val: Epoch = {} | Loss {:.3f} | Accuracy = {:.3f}'.format(epoch + 1, avg_val_loss, val_accuracy))
            if val_accuracy > min_accuracy:
                print("[||| NEW BEST on val||||]")
                save_path_model = os.path.join(model_folder, model_checkpoint)
                torch.save(model.state_dict(), save_path_model)
                min_accuracy = val_accuracy
"""
    optim_scheduler.step()


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-13-d1c997c7d697>", line 31, in __getitem__
    image = self.transforms(image)
  File "/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py", line 95, in __call__
    img = t(img)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
 